In [5]:
from libSOGAupdate import *
from libSOGAshared import *
from ASGMTListener import *
from ASGMTParser import * 
from ASGMTLexer import *

dist = Dist(['x', 'y'], GaussianMix([torch.tensor(1.)], [torch.tensor([1., 2.])], [torch.eye(2)]))
expr = 'x = 3*x + 3*y + 6*gm([0.5, 0.5], [0.,1.], [1., 0.5])'
data = {}

update_rule(dist, expr, data)

Dist<['x', 'y'],pi: [tensor(0.5000), tensor(0.5000)]  mu: [tensor([9., 2.]), tensor([15.,  2.])] sigma: [tensor([[54.,  3.],
        [ 3.,  1.]]), tensor([[27.,  3.],
        [ 3.,  1.]])]>